# Exploring, cleaning and enriching datas

# 1. Exploring

In [46]:
import numpy as np
import pandas as pd
import src.clean_utils as cu
import requests
from bs4 import BeautifulSoup

In [2]:
# Importing Data, we know thw topic but. What tipe of datas do we have?
Abb_NY = pd.read_csv("data/AB_NYC_2019.csv",encoding = "ISO-8859-1")

In [3]:
Abb_NY.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [4]:
#We can se than we have a very bit Dataset but, would it be too dirty?
Abb_NY.shape

(48895, 16)

In [5]:
Abb_NY.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [6]:
# It seems I found a very clean data set. Where only 2 columns  have over 20% of nulls datas but they don`t affect to our study.
perc_null_col = Abb_NY.isnull().sum().apply(lambda x: x/Abb_NY.shape[0]).sort_values(ascending=False)
perc_null_col

reviews_per_month                 0.205583
last_review                       0.205583
host_name                         0.000429
name                              0.000327
availability_365                  0.000000
calculated_host_listings_count    0.000000
number_of_reviews                 0.000000
minimum_nights                    0.000000
price                             0.000000
room_type                         0.000000
longitude                         0.000000
latitude                          0.000000
neighbourhood                     0.000000
neighbourhood_group               0.000000
host_id                           0.000000
id                                0.000000
dtype: float64

In [7]:
# And what about our rows? there are a few with missing datas, but as we saw this datas are fron review column and do not affect us.
perc_null_row=Abb_NY.isnull().sum(axis=1).apply(lambda x: x/Abb_NY.shape[1]).sort_values(ascending=False)
perc_null_row

16071    0.1875
27777    0.1875
38992    0.1875
6605     0.1875
2854     0.1875
          ...  
29611    0.0000
29610    0.0000
29608    0.0000
29607    0.0000
0        0.0000
Length: 48895, dtype: float64

In [8]:
# There are not duplicate visits:
Abb_NY.duplicated().sum()

0

In [9]:
# It looks like we have both numerical and categorical data
Abb_NY_dtype=Abb_NY.dtypes
Abb_NY_dtype

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

After this review and exploring of data, we can conclude we have a very good dataframe, with clean data, quantity to be analyze and some columns we can drop and will  not affect our conclusions.

# 2. Cleaning de Data

In [10]:
# We remenber the columns we have in order to drop those we dont need
Abb_NY.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [11]:
drop_cols = ["id","name","host_id","host_name","latitude", "longitude","reviews_per_month","calculated_host_listings_count","availability_365"]
Abb_NY_clean = Abb_NY.drop(drop_cols, axis =1)
Abb_NY_clean

,neighbourhood_group,neighbourhood,room_type,price,minimum_nights,number_of_reviews,last_review
0,Brooklyn,Kensington,Private room,149,1,9,2018-10-19
1,Manhattan,Midtown,Entire home/apt,225,1,45,2019-05-21
2,Manhattan,Harlem,Private room,150,3,0,NaN
3,Brooklyn,Clinton Hill,Entire home/apt,89,1,270,2019-07-05
4,Manhattan,East Harlem,Entire home/apt,80,10,9,2018-11-19
...,...,...,...,...,...,...,...
48890,Brooklyn,Bedford-Stuyvesant,Private room,70,2,0,NaN
48891,Brooklyn,Bushwick,Private room,40,4,0,NaN
48892,Manhattan,Harlem,Entire home/apt,115,10,0,NaN
48893,Manhattan,Hell's Kitchen,Shared room,55,1,0,NaN


In [12]:
Abb_NY_clean["last_review"].value_counts(dropna=False)

NaN           10052
2019-06-23     1413
2019-07-01     1359
2019-06-30     1341
2019-06-24      875
              ...  
2016-03-07        1
2014-08-24        1
2015-05-03        1
2016-02-10        1
2015-02-05        1
Name: last_review, Length: 1765, dtype: int64

In [13]:
Abb_NY_clean["year"] = Abb_NY_clean.last_review.dropna().apply(cu.extract_year)

In [14]:
Abb_NY_clean.head()

,neighbourhood_group,neighbourhood,room_type,price,minimum_nights,number_of_reviews,last_review,year
0,Brooklyn,Kensington,Private room,149,1,9,2018-10-19,2018.0
1,Manhattan,Midtown,Entire home/apt,225,1,45,2019-05-21,2019.0
2,Manhattan,Harlem,Private room,150,3,0,NaN,NaN
3,Brooklyn,Clinton Hill,Entire home/apt,89,1,270,2019-07-05,2019.0
4,Manhattan,East Harlem,Entire home/apt,80,10,9,2018-11-19,2018.0


In [15]:
Abb_NY_clean["year"].value_counts(dropna=False)

2019.0    25209
NaN       10052
2018.0     6050
2017.0     3205
2016.0     2707
2015.0     1393
2014.0      199
2013.0       48
2012.0       25
2011.0        7
Name: year, dtype: int64

In [16]:
drop_cols = ["last_review"]
Abb_NY_clean = Abb_NY_clean.drop(drop_cols, axis =1)
Abb_NY_clean

,neighbourhood_group,neighbourhood,room_type,price,minimum_nights,number_of_reviews,year
0,Brooklyn,Kensington,Private room,149,1,9,2018.0
1,Manhattan,Midtown,Entire home/apt,225,1,45,2019.0
2,Manhattan,Harlem,Private room,150,3,0,NaN
3,Brooklyn,Clinton Hill,Entire home/apt,89,1,270,2019.0
4,Manhattan,East Harlem,Entire home/apt,80,10,9,2018.0
...,...,...,...,...,...,...,...
48890,Brooklyn,Bedford-Stuyvesant,Private room,70,2,0,NaN
48891,Brooklyn,Bushwick,Private room,40,4,0,NaN
48892,Manhattan,Harlem,Entire home/apt,115,10,0,NaN
48893,Manhattan,Hell's Kitchen,Shared room,55,1,0,NaN


In [17]:
Abb_NY_clean["year"] = Abb_NY_clean[Abb_NY_clean["year"]>2015]["year"]
Abb_NY_clean

,neighbourhood_group,neighbourhood,room_type,price,minimum_nights,number_of_reviews,year
0,Brooklyn,Kensington,Private room,149,1,9,2018.0
1,Manhattan,Midtown,Entire home/apt,225,1,45,2019.0
2,Manhattan,Harlem,Private room,150,3,0,NaN
3,Brooklyn,Clinton Hill,Entire home/apt,89,1,270,2019.0
4,Manhattan,East Harlem,Entire home/apt,80,10,9,2018.0
...,...,...,...,...,...,...,...
48890,Brooklyn,Bedford-Stuyvesant,Private room,70,2,0,NaN
48891,Brooklyn,Bushwick,Private room,40,4,0,NaN
48892,Manhattan,Harlem,Entire home/apt,115,10,0,NaN
48893,Manhattan,Hell's Kitchen,Shared room,55,1,0,NaN


In [18]:
Abb_NY_clean = Abb_NY_clean[Abb_NY_clean["year"].notnull()]
Abb_NY_clean

,neighbourhood_group,neighbourhood,room_type,price,minimum_nights,number_of_reviews,year
0,Brooklyn,Kensington,Private room,149,1,9,2018.0
1,Manhattan,Midtown,Entire home/apt,225,1,45,2019.0
3,Brooklyn,Clinton Hill,Entire home/apt,89,1,270,2019.0
4,Manhattan,East Harlem,Entire home/apt,80,10,9,2018.0
5,Manhattan,Murray Hill,Entire home/apt,200,3,74,2019.0
...,...,...,...,...,...,...,...
48782,Manhattan,Upper East Side,Private room,129,1,1,2019.0
48790,Queens,Flushing,Private room,45,1,1,2019.0
48799,Staten Island,Great Kills,Private room,235,1,1,2019.0
48805,Bronx,Mott Haven,Entire home/apt,100,1,2,2019.0


In [19]:
Abb_NY_clean["year"].value_counts(dropna=False)

2019.0    25209
2018.0     6050
2017.0     3205
2016.0     2707
Name: year, dtype: int64

# 2. Enriching de Data

In [47]:
url_NYmonuments = "https://en.wikipedia.org/wiki/List_of_buildings,_sites,_and_monuments_in_New_York_City"

In [48]:
response = requests.get(url_NYmonuments)

In [49]:
response

<Response [200]>

In [50]:
soup = BeautifulSoup(response.content)

In [51]:
NYmonuments_tags = soup.find_all(
    name="div", 
    class_="div-col"
)

In [52]:
len(NYmonuments_tags)

3

In [53]:
NYmonuments_tags[0]

<div class="div-col" style="column-width: 30em;">
<ul><li><a href="/wiki/American_Museum_of_Natural_History" title="American Museum of Natural History">American Museum of Natural History</a> (<a href="/wiki/Manhattan" title="Manhattan">Manhattan</a>)
<ul><li><a href="/wiki/Rose_Center_for_Earth_and_Space" title="Rose Center for Earth and Space">Rose Center for Earth and Space</a></li></ul></li>
<li><a href="/wiki/America%27s_Response_Monument" title="America's Response Monument">America's Response Monument</a> (<a href="/wiki/Manhattan" title="Manhattan">Manhattan</a>)</li>
<li><a href="/wiki/Apollo_Theater" title="Apollo Theater">Apollo Theater</a> (<a href="/wiki/Manhattan" title="Manhattan">Manhattan</a>)</li>
<li><a class="mw-redirect" href="/wiki/Bank_of_America_Tower_(New_York_City)" title="Bank of America Tower (New York City)">Bank of America Tower</a> (<a href="/wiki/Manhattan" title="Manhattan">Manhattan</a>)</li>
<li><a class="mw-redirect" href="/wiki/Battery_Park" title="Ba

In [54]:
NYmonuments_tags_text = [monument.text for monument in NYmonuments_tags]
NYmonuments_tags_text

["\nAmerican Museum of Natural History (Manhattan)\nRose Center for Earth and Space\nAmerica's Response Monument (Manhattan)\nApollo Theater (Manhattan)\nBank of America Tower (Manhattan)\nBattery Park (Manhattan)\nCastle Clinton\nEast Coast Memorial\nThe Sphere\nBattery Park City Ferry Terminal (Manhattan)\nBrill Building (Manhattan)\nBronx Zoo (Bronx)\nBrooklyn Botanic Garden (Brooklyn)\nBrooklyn Bridge (connects Brooklyn and Manhattan)\nBrooklyn Museum (Brooklyn)\nBrooklyn Public Library (Brooklyn)\nBryant Park (Manhattan)\nCarnegie Hall (Manhattan)\nCathedral of St. John the Divine (Manhattan)\nCentral Park (Manhattan)\nCentral Park Zoo\nStrawberry Fields\nChelsea Piers (Manhattan)\nChrysler Building (Manhattan)\nCiti Field (Queens)\nCleopatra's Needle\nCollege Point Little League Building (Queens)\nColumbia University (Manhattan)\nColumbus Circle (Manhattan)\nDowntown Athletic Club (Manhattan)\nEast River Park (Manhattan)\nEast River State Park (Brooklyn)\nEleanor Roosevelt Monume

In [55]:
len(NYmonuments_tags_text)

3

In [56]:
NYmonuments_tags_text = NYmonuments_tags_text[0]

In [1]:
NYmonuments_tags_list=[]
for name in NYmonuments_tags_text:
    NYmonuments_tags_list.append(name.replace("\n",","))
NYmonuments_tags_list

NameError: name 'NYmonuments_tags_text' is not defined